In [1]:
import numpy as np
import pandas as pd
import os
import warnings
import librosa
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from random import shuffle
import scipy.stats as st

from pynwb import NWBHDF5IO
from pynwb.image import ImageSeries
from pynwb.epoch import TimeIntervals

In [3]:
# ----- SET UP FILE HANDLING -----

computer = 'Dorian'

match computer:
    case 'Dorian':
        nwbpath = 'M:\\scn2a-paper-GWJSNH\\NWB-files'
        
    case 'Ernest':
        nwbpath = 'C:\\Users\\nhogl\\Documents\\GitHub\\scn2a-paper-GWJSNH-code\\NWB'

assay = 'timedMating'
allnwb = []
allnwb += [os.path.join(nwbpath,each) for each in os.listdir(nwbpath) if assay in each]

figp=os.path.join('figures','2024July') # where to save figures
saveplots=False

usefield = 'single_behavior_Mount'

In [ ]:
# ---- Look at count of mounts in timed matings ----

# keep track of details in arrays
allGT = []
allSex = []
times = []
colors = []
nroccurrences = []
# behav='Mount'

# loop over nwb files matching specified assay name
for i,fname in enumerate(allnwb):
    # open file
    io = NWBHDF5IO(os.path.join(nwbpath,fname), mode="r") # <-- open in write to append snippets?
    nwbfile = io.read()
    
    # for sanity check, print out the name of the file being handled
    pathsplit = fname.split('\\')
    recstem = pathsplit[-1].strip('.nwb')
    print(recstem)
    
    # filter out files that don't have complete timelines
    
    # keep track of metadata
    allSex.append(nwbfile.subject.sex)
    allGT.append(nwbfile.subject.genotype)
    colors.append(nwbfile.lab_meta_data['vole_metadata'].colors[:])
       
    # -- Get time of first behavior --
    
    # load up the BORIS table from the NWB file
    statsdf = nwbfile.intervals[usefield].to_dataframe()
    
    # find start time of assay
    firstBehav = np.min(statsdf.start_time)
    times.append(firstBehav)
    # count total number of mounts
    nroccurrences.append(statsdf.shape[0])